# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [177]:
# import libraries
import os
import re
import numpy as np
import pandas as pd
import pickle
from time import time

# import nltk libraries
import nltk
nltk.download(['punkt', 'wordnet', 'stopwords'])
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords

# import sklearn libraries
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer, TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report, f1_score, make_scorer, accuracy_score, balanced_accuracy_score, fbeta_score
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV 
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import LinearSVC
from sklearn.multioutput import MultiOutputClassifier


from sqlalchemy import create_engine

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Peter\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Peter\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Peter\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [178]:
# load data from database
engine = create_engine('sqlite:///DisasterResponse.db')
df = pd.read_sql_table('disaster_table', con=engine)

X = df['message'] # reads message column
y = df.iloc[:,4:] # alls row, from column 4 to all col

In [179]:
X.head()

0    Weather update - a cold front from Cuba that c...
1              Is the Hurricane over or is it not over
2                      Looking for someone but no name
3    UN reports Leogane 80-90 destroyed. Only Hospi...
4    says: west side of Haiti, rest of the country ...
Name: message, dtype: object

In [180]:
# displays length of characters in each message
df['message'] = df['message'].apply(lambda x: len(x))
df['message']

0         66
1         39
2         31
3        100
4         63
        ... 
26211    148
26212    136
26213    141
26214    191
26215    209
Name: message, Length: 26216, dtype: int64

In [181]:
# index of column names
y.columns

Index(['related', 'request', 'offer', 'aid related', 'medical help',
       'medical products', 'search and rescue', 'security', 'military',
       'child alone', 'water', 'food', 'shelter', 'clothing', 'money',
       'missing people', 'refugees', 'death', 'other aid',
       'infrastructure related', 'transport', 'buildings', 'electricity',
       'tools', 'hospitals', 'shops', 'aid centers', 'other infrastructure',
       'weather related', 'floods', 'storm', 'fire', 'earthquake', 'cold',
       'other weather', 'direct report'],
      dtype='object')

### 2. Write a tokenization function to process your text data

In [182]:
url_regex = 'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'

# function to detect url, normalize, tokenize, remove stopwords, lemmatize words 
def tokenize(text, lemmatizer=WordNetLemmatizer()):
    # Detect urls with url_regex
    detected_urls = re.findall(url_regex, text)
    for url in detected_urls:
        text = text.replace(url, "urlplaceholder")

    # Normalizes and tokenizes (removes punctuations and converts to lowercase)
    tokens = nltk.word_tokenize(re.sub(r"[^a-zA-Z0-9]", " ", text.lower()))

    # lemmatize and remove stop words
    clean_tokens = []
    for tok in tokens:
        clean_tok = lemmatizer.lemmatize(tok).lower().strip()
        if clean_tok not in stopwords.words("english"):
            clean_tokens.append(clean_tok)
    pass
    return clean_tokens

In [183]:
# Converts text in X to word count vectors (takes long)
vect = CountVectorizer(tokenizer=tokenize)
X_vect = vect.fit_transform(X)

In [184]:
# Converts word and count into a Pandas dataframe
line , num = [], [] #create empty list
for l, n in vect.vocabulary_.items(): # 
    line.append(l)
    num.append(n)

df_count = pd.DataFrame.from_dict({'word': line, 'count': num})

In [185]:
df_count

,word,count
0,weather,30214
1,update,29355
2,cold,6334
3,front,11379
4,cuba,7410
...,...,...
31115,ramu,22757
31116,chokoria,5970
31117,teargassed,27491
31118,harcourt,12702


### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [186]:
pipeline = Pipeline([
    ('vect', TfidfVectorizer(tokenizer=tokenize)),
    ('clf', MultiOutputClassifier(RandomForestClassifier(n_estimators=100, n_jobs =-1)))
])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [187]:
# Splits dataset into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state = 42)

In [144]:
# Fits pipeline on train set (takes long)
pipeline.fit(X_train, y_train)

Pipeline(steps=[('vect',
                 TfidfVectorizer(tokenizer=<function tokenize at 0x000001D10F01BD30>)),
                ('clf',
                 MultiOutputClassifier(estimator=RandomForestClassifier(n_jobs=-1)))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [156]:
# Predit pipeline X_test (takes long)
y_pred_train = pipeline.predict(X_train)
y_pred = pipeline.predict(X_test)

In [165]:
from sklearn.multioutput import MultiOutputClassifier

# Print classification report on test data
from sklearn.metrics import classification_report, accuracy_score
for i in range(y_test.shape[1]):
    print("Category: {}".format(y.columns[i]))
    print(classification_report(y_test[:,i], y_pred[:,i]))
        ))

### 6. Improve your model
Use grid search to find better parameters. 

In [ ]:
from sklearn.model_selection import GridSearchCV

parameters = {
        'features__text_pipeline__vect__ngram_range': ((1, 1), (1, 2)),
        'features__text_pipeline__vect__max_df': (0.5, 0.75, 1.0),
        'features__text_pipeline__vect__max_features': (None, 5000, 10000),
        'features__text_pipeline__tfidf__use_idf': (True, False),
        'clf__n_estimators': [50, 100, 200],
        'clf__min_samples_split': [2, 3, 4],
        'features__transformer_weights': (
            {'text_pipeline': 1, 'starting_verb': 0.5},
            {'text_pipeline': 0.5, 'starting_verb': 1},
            {'text_pipeline': 0.8, 'starting_verb': 1},
        )
    }

cv = GridSearchCV(pipeline, param_grid=parameters, verbose=3, n_jobs = 10)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

### 9. Export your model as a pickle file

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.